In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 500)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, hp, tpe, space_eval

from sklearn.model_selection import KFold, TimeSeriesSplit
import lightgbm as lgb
from time import time
from tqdm import tqdm_notebook

from xgboost import XGBClassifier
import os

import gc
import warnings
warnings.filterwarnings('ignore')

/Users/didi/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning:

Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.



In [2]:
# NROWS = None
NROWS = 2000

In [3]:
# train_identity = pd.read_csv('../input/train_identity.csv', nrows=NROWS)
# train_transaction = pd.read_csv('../input/train_transaction.csv', nrows=NROWS)
# train = train_transaction.merge(train_identity, how='left', on='TransactionID')

In [4]:
# train.to_csv('../temp/train_temp' + str(NROWS) + '.csv', index=False, header=True)

In [5]:
train = pd.read_csv('../temp/train_temp' + str(NROWS) + '.csv')

In [6]:
train['card_str'] = train["card1"].apply(lambda x: str(x)) + "_" + train["card2"].apply(lambda x: str(x)) +\
                "_" + train["card3"].apply(lambda x: str(x)) + "_" + train["card4"].apply(lambda x: str(x)) +\
                "_" + train["card5"].apply(lambda x: str(x)) + "_" + train["card6"].apply(lambda x: str(x))

In [7]:
train.loc[train["isFraud"] == 1,  ["TransactionID", "card_str", "isFraud"]]

,TransactionID,card_str,isFraud
203,2987203,18268_583.0_150.0_visa_226.0_credit,1
240,2987240,13413_103.0_185.0_visa_137.0_credit,1
243,2987243,13413_103.0_185.0_visa_137.0_credit,1
245,2987245,13413_103.0_185.0_visa_137.0_credit,1
288,2987288,16578_545.0_185.0_visa_226.0_credit,1
367,2987367,4425_562.0_150.0_mastercard_197.0_credit,1
405,2987405,4504_500.0_185.0_mastercard_219.0_credit,1
630,2987630,5812_408.0_185.0_mastercard_224.0_debit,1
683,2987683,5812_408.0_185.0_mastercard_224.0_debit,1
736,2987736,15063_nan_150.0_visa_226.0_credit,1


当前刷卡和前一笔刷卡信息(金额, card, )是否一致
最近1/5/10/50/100单和当前信息一致的个数
最近的1单

In [13]:
delta = 3
train["window_delta_" + str(delta)] = 0

train[["TransactionID", "isFraud", "card_str", "window_delta_" + str(delta)]].head()

,TransactionID,isFraud,card_str,window_delta_3
0,2987000,0,13926_nan_150.0_discover_142.0_credit,0
1,2987001,0,2755_404.0_150.0_mastercard_102.0_credit,0
2,2987002,0,4663_490.0_150.0_visa_166.0_debit,0
3,2987003,0,18132_567.0_150.0_mastercard_117.0_debit,0
4,2987004,0,4497_514.0_150.0_mastercard_102.0_credit,0


In [14]:
for i in tqdm_notebook(range(delta, len(train))):
    cur = i
    start = cur - delta
    end =   cur + delta
    cur_card_str = train.loc[cur, "card_str"]
    cnt  = train.loc[start:end].loc[train["card_str"] == cur_card_str].shape[0] - 1
    train.loc[cur, "window_delta_" + str(delta)] = cnt

In [22]:
features = ["TransactionID", "card_str", "window_delta_" + str(delta), "isFraud"]

In [15]:
train.loc[train["isFraud"] == 1,  ["TransactionID", "card_str", "window_delta_" + str(delta), "isFraud"]]

,TransactionID,card_str,window_delta_3,isFraud
203,2987203,18268_583.0_150.0_visa_226.0_credit,0,1
240,2987240,13413_103.0_185.0_visa_137.0_credit,1,1
243,2987243,13413_103.0_185.0_visa_137.0_credit,2,1
245,2987245,13413_103.0_185.0_visa_137.0_credit,1,1
288,2987288,16578_545.0_185.0_visa_226.0_credit,0,1
367,2987367,4425_562.0_150.0_mastercard_197.0_credit,0,1
405,2987405,4504_500.0_185.0_mastercard_219.0_credit,0,1
630,2987630,5812_408.0_185.0_mastercard_224.0_debit,0,1
683,2987683,5812_408.0_185.0_mastercard_224.0_debit,0,1
736,2987736,15063_nan_150.0_visa_226.0_credit,0,1


In [23]:
train.loc[(train["isFraud"] == 1) & (train["window_delta_" + str(delta)] != 0), features]

,TransactionID,card_str,window_delta_3,isFraud
240,2987240,13413_103.0_185.0_visa_137.0_credit,1,1
243,2987243,13413_103.0_185.0_visa_137.0_credit,2,1
245,2987245,13413_103.0_185.0_visa_137.0_credit,1,1
780,2987780,8732_360.0_150.0_mastercard_229.0_debit,1,1
781,2987781,8732_360.0_150.0_mastercard_229.0_debit,1,1


In [24]:
train.loc[(train["isFraud"] == 0) & (train["window_delta_" + str(delta)] != 0), features]

,TransactionID,card_str,window_delta_3,isFraud
141,2987141,8194_325.0_150.0_visa_195.0_debit,1,0
144,2987144,8194_325.0_150.0_visa_195.0_debit,1,0
179,2987179,15497_490.0_150.0_visa_226.0_debit,1,0
182,2987182,15497_490.0_150.0_visa_226.0_debit,1,0
218,2987218,12469_360.0_150.0_mastercard_166.0_debit,1,0
221,2987221,12469_360.0_150.0_mastercard_166.0_debit,1,0
300,2987300,2115_415.0_185.0_mastercard_195.0_credit,1,0
302,2987302,2115_415.0_185.0_mastercard_195.0_credit,2,0
305,2987305,2115_415.0_185.0_mastercard_195.0_credit,1,0
332,2987332,9500_321.0_150.0_visa_226.0_debit,1,0
